<a href="https://colab.research.google.com/github/sarath-githu/HAR_LSTM_COLAB/blob/master/akshay_gupta123_cattle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.2/614.2 kB 7.2 MB/s eta 0:00:00


In [6]:
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt


In [3]:
import torch

In [10]:
import argparse
import cv2
import numpy as np
from pathlib import Path
import torch
import time
import sys
sys.path.append("/content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/util_function")
from plot import colors, plot_one_box
from misc import set_logging, LoadImages, increment_path

def detect(opt):
    source, weights, view_img, save_txt = opt.source, opt.weights, opt.view_img, opt.save_txt
    save_img = not opt.nosave and not source.endswith('.txt')  # save inference images

    # Directories
    save_dir = increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()

    # Load model
    try:
        model = torch.hub.load('ultralytics/yolov5','custom', path_or_model="/content/300.pt",force_reload=True)
    except :
        model = torch.hub.load('ultralytics/yolov5','custom',"/content/300.pt",force_reload=True)

    model.conf = opt.conf_thres
    model.iou = opt.iou_thres
    names = model.module.names if hasattr(model, 'module') else model.names  # get class names

    # Set Dataloader
    vid_path, vid_writer = None, None
    dataset = LoadImages("/content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets")

    # Run inference
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:

        # Inference
        t1 = time.time()
        pred = model(im0s).xyxy
        t2 = time.time()


        # Process detections
        for i, det in enumerate(pred):  # detections per image
            p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[:2]  # print string
            imc = im0
            if len(det):

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        #xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xyxy, conf) if opt.save_conf else (cls, *xyxy)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img  or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if opt.hide_labels else (names[c] if opt.hide_conf else f'{names[c]} {conf:.2f}')
                        plot_one_box(xyxy, im0, label=label,color=colors(c, True), line_thickness=opt.line_thickness)

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                            print("1")
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'

                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default='300.pt', help='model.pt path(s)')
    parser.add_argument('--source', type=str, default='data', help='source')
    parser.add_argument('--conf-thres', type=float, default=0.15, help='object confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.35, help='IOU threshold for NMS')
    parser.add_argument('--view-img', action='store_true', help='display results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--project', default='runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument("-f",required=False)
    opt = parser.parse_args()
    print(opt)

    with torch.no_grad():
        detect(opt=opt)


Namespace(weights='300.pt', source='data', conf_thres=0.15, iou_thres=0.35, view_img=False, save_txt=False, save_conf=False, line_thickness=3, hide_labels=False, hide_conf=False, nosave=False, project='runs/detect', name='exp', exist_ok=False, f='/root/.local/share/jupyter/runtime/kernel-42decb8f-2957-4feb-964d-791fd1b9f55a.json')


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-9-10 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 224 layers, 7053910 parameters, 0 gradients
Adding AutoShape... 


image 1/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/1.jpg: 480x480 Done. (0.460s)
image 2/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/10.jpg: 480x480 9 cows, Done. (0.463s)
image 3/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/2.jpg: 480x480 1 cow, Done. (0.461s)
image 4/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/3.jpg: 480x480 6 cows, Done. (0.452s)
image 5/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/4.jpg: 480x480 5 cows, Done. (0.460s)
image 6/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/5.jpg: 480x480 2 cows, Done. (0.449s)
image 7/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/6.jpg: 480x480 1 cow, Done. (0.467s)
image 8/10 /content/drive/MyDrive/Cattle-detection-main/Cattle-detection-main/assets/7.jpg: 480x480 7 cows, Done. (0.467s)
image 9/10 /content/drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


"